## Packages

In [ ]:
import random
import numpy as np
import os
import torch
import torch.nn as nn
import pandas as pd 
from torch.utils.data import Dataset, DataLoader
import pickle
from sklearn import metrics
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle
from scipy import stats
import seaborn as sns
import math
import matplotlib as mpl

In [ ]:
from tqdm import tqdm_notebook, trange

def seed_everything(seed = 42): 
  random.seed(seed) 
  os.environ['PYTHONHASHSEED'] = str(seed) 
  np.random.seed(seed)
  torch.manual_seed(seed) 
  torch.cuda.manual_seed(seed) 
  torch.backends.cudnn.deterministic = True
# For reproducible results
seed_everything()

In [ ]:
mpl.style.use('seaborn')

In [ ]:
''' using fastai models'''
# !pip install -U fastai 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
%cd /content/gdrive/My Drive/seq

## Data Preprocessing

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, Y):
        self.data = X
        self.target = Y
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [ ]:
npzfile = np.load('data/gtex/adi.npz')

In [ ]:
X, Y = npzfile['arr_0'], npzfile['arr_1']

In [ ]:
X, Y = shuffle(X, Y, random_state = 0)

In [ ]:
testX = X[int(len(Y)*0.8):]
testY = Y[int(len(Y)*0.8):]
validX = X[int(len(Y)*0.6):int(len(Y)*0.8)]
validY = Y[int(len(Y)*0.6):int(len(Y)*0.8)]
trainX = X[:int(len(Y)*0.6)]
trainY = Y[:int(len(Y)*0.6)]

### Convert to Torch Data

In [ ]:
train_X = torch.from_numpy(trainX)
train_y = torch.from_numpy(trainY)
valid_X  = torch.from_numpy(validX)
valid_y = torch.from_numpy(validY)
test_X = torch.from_numpy(testX)
test_y = torch.from_numpy(testY)

In [ ]:
train_dataset = MyDataset(train_X, train_y)
valid_dataset = MyDataset(valid_X, valid_y)
test_dataset = MyDataset(test_X, test_y)

## Helper Functions

In [ ]:
def bestmodel(model_name,save_model_time,valid_loss):
    bestloss = 10000
    if valid_loss < bestloss :
        bestloss = valid_loss
        torch.save(model_name, 'model/model{save_model_time}/bestmodel.pkl'.format(save_model_time=save_model_time))
        torch.save(model_name.state_dict(), 'model/model{save_model_time}/net_params_bestmodel.pkl'.format(save_model_time=save_model_time))
    return True  

### Mutation

In [ ]:
def reverse(x):
    ori = []
    all_cap_letters = ['A', 'T', 'G', 'C'] # depends on dataset
    all_cap_letters = ['A', 'C', 'G', 'T']

    for c in range(len(x[0])):
        for r in range(4):
            if x[r][c] == 1:
                ori.append(all_cap_letters[r])

    return ''.join(ori)

In [ ]:
def onehot(X):
    X_onehot = np.zeros((4, len(X)), dtype=np.float32)
    all_cap_letters = ['A', 'T', 'G', 'C']  # depends on dataset
    all_cap_letters = ['A', 'C', 'G', 'T']

    for li in range(len(X)):
        letter = X[li]
        X_onehot[all_cap_letters.index(str(letter))][li] = 1
        #X_onehot[li][all_cap_letters.index(str(letter))] = 1

    return X_onehot

In [ ]:
# sample='GGTAAACAATAATAAAAAGCCTTTGAATCTGACTTCCCTTTTGACGAGTATAAACGTACTTCAATTAGAGCTCACTGCCTTGACATAGTTCCTCCTTCAGACTTAGTCCCTGTCACAAACCCCTGCCTGGCCTGTCACAGACCCTTTACTTTTTATCTAGTTCTTACGGTCTTTACTCATGGGGACTCTCTTAACAGCCTACACAGAGTTCCCCTACCCAGCTCTAACAGCGGTGACCCTTGCTGTGGATTCTTCGCCCCACCTGCTCCCACGAGGGCACCCAGCTTGTGAAAAACCAGCTTAACCCAGATTTCTGTTCATCCATCACTCCTGCTCGCAGCGTCACCTGAAGGAGTAATCTCAGCCTCACTCGGGCAGCTTTGGGTTTTAAAAATCAATGCTGTAAAAGTGAACATTGCAGTTTCTCTGACTCCACAGCAAGTCAACTTCAACTACACTCAAGCTAACTAGGCTATGGAGCGTTTGCCGACCTCATTTACCAGGGGAACCGAATACTCTATCTTTCAATGCTAACTTTCCTGTCCTACAGTACTTCTGGAGCACTGAAAACTCCCCTAACACTCCTCTTCCCCTACCTCTGCTCTAACCTTCTCAACCAGGGGCTGTCGGTCTGGTACTCTCAAGCTGCCTCTTCCTGCAGCCATTCGAACAGTGCCCCCTAGTGGCCAGTGTTTCAAGTGCAATTGAAGAGGCTTGCAAGACTCATGCCACTGAGTCAGCTTAATTCAACATCTCTATGTGCCCAAGACTTCTCCCAGGGATATTCATTTTTAACCAGGACTATCAGTTCTTTATCCTACAAAGGCATAAGGGTAAATGAAACCAGGAGCAGACAATAAGGATATTGAGGGTTTTTATAACCTGGGCAATGGCCTGATCAAAGACTACTCAGCTTAGTACTAGCTAAACGATACTATCAGCAGTCCAGTCACCATTTCCTATAAACATCAGAAGCAGGAAGCTGTGTGGCTCTGTCATTT'

In [ ]:
def mutation(loader, model):
    dic = {0: [1,2,3], 1: [0, 2, 3], 2:[0,1,3], 3:[0,1,2]}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    res = []
    sample = []
    for m, (data, labels) in enumerate(loader):
        tmp = data.tolist()
        array = np.array(tmp[0])
        sample = np.transpose(data[0])
        #sample = sample[200:1001-200]
        #for i in range(669, 690): # mutating on most enriched
        for i in range(len(array[0])):
          if i >= 516 and i <= 526:
            idx = np.where(array[:,i]==1)
            mut = []
            for val in dic[idx[0][0]]:
              new_letter = []
              for j in range(4):
                if j == val:
                  new_letter.append(1.0)
                else:
                  new_letter.append(0.0)
              v = np.array(new_letter)
              v = v.reshape((4,1))
              new_data = np.hstack((array[:,:i], v))
              new_data = np.hstack((new_data, array[:,i+1:]))
              new_data = torch.from_numpy(np.array([new_data], dtype=np.float32))
              new_set = MyDataset(new_data,labels)
              new_loader = torch.utils.data.DataLoader(new_set, batch_size=1)
              for k, (d, l) in enumerate(new_loader):
                d, l = d.to(device),l.to(device)
                with torch.no_grad():
                    output = model(d)
                # make our predictions
                predicted = output.tolist()[0][0]
                # mut.append(abs(math.log(abs(predicted/(l.tolist()[0])))))
                mut.append(abs(abs(predicted-l.tolist()[0])/l.tolist()[0])+1)
            res.append(max(mut))
          else:
            res.append(1)
    return res, sample

In [ ]:
''' same function as above but slower runtime '''
# def mutation(dataset, model):
#   dic = {'A':['C','G', 'T'], 'C':['A','G','T'], 'G':['A','C','T'], 'T':['A','C','G']}
#   seq = reverse(dataset.data[0])
#   val = dataset.target[0]
#   res = []
#   for i in range(len(seq)):
#     choices = dic[seq[i]]
#     tmp = []
#     for c in choices:
#       tmp_s = seq[:i] + c + seq[i+1:]
#       tmp.append(onehot(tmp_s))
#     tmp_d = torch.from_numpy(np.array(tmp, dtype=np.float32))
#     tmp_v = torch.from_numpy(np.array([val, val, val]))
#     tmp_set = MyDataset(tmp_d, tmp_v)
#     loader = torch.utils.data.DataLoader(tmp_set, batch_size=1)
#     t, p = get_list_con(model, loader)
#     tmp_max = 0
#     for i in range(3):
#       tmp_max = max(tmp_max, abs(p[i]-t[i]))
#     res.append(tmp_max)
#   return res

In [ ]:
def weight_matrix(model):
  out, input = mutation(mut_loader, model)
  #weighted = np.zeros([601,4])
  weighted = np.zeros([1001, 4])
  for i in range(len(out)):
    weighted[i] = out[i]*input[i]
  return weighted

### Heat Map

In [ ]:
def mutation_each(loader, model, letter):
    dic = {0: [letter], 1: [letter], 2:[letter], 3:[letter]}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    res = []
    sample = []
    for m, (data, labels) in enumerate(loader):
        tmp = data.tolist()
        array = np.array(tmp[0])
        sample = array[:,200:len(array[0])-200]
        for i in range(200, len(array[0])-200): # mutating on 600bp
          idx = np.where(array[:,i]==1)
          mut = []
          for val in dic[idx[0][0]]:
            new_letter = []
            for j in range(4):
              if j == val:
                new_letter.append(1.0)
              else:
                new_letter.append(0.0)
            v = np.array(new_letter)
            v = v.reshape((4,1))
            new_data = np.hstack((array[:,:i], v))
            new_data = np.hstack((new_data, array[:,i+1:]))
            new_data = torch.from_numpy(np.array([new_data], dtype=np.float32))
            new_set = MyDataset(new_data,labels)
            new_loader = torch.utils.data.DataLoader(new_set, batch_size=1)
            for k, (d, l) in enumerate(new_loader):
              d, l = d.to(device),l.to(device)
              with torch.no_grad():
                  output = model(d)
             # make our predictions
            predicted = 0
            for o in output.tolist():
              predicted = (o[0])
              res.append((predicted/l).tolist()[0])
    return res

In [ ]:
def heat(loader, model):
  matrix = []
  for i in range(4):
    res = mutation_each(loader, model, i)
    matrix.append(res[250:350])
  matrix = np.array(matrix)
  fig = plt.figure(figsize=(40, 5))
  ax = sns.heatmap(matrix, cmap="YlGnBu",  yticklabels=['A','C','G','T'])
  fig.show()

### Loss Functions

In [ ]:
def negative_binomial_layer(x):
    n,p = torch.unbind(x, dim=1)
    n,p = [torch.unsqueeze(t,dim=-1) for t in (n,p)]
    n = torch.nn.functional.softplus(n)
    p = torch.sigmoid(p)
    x = torch.cat((n,p), axis=1)
    return x

In [ ]:
def negative_binomial_dist(x):
    r, p = torch.unbind(x, dim=1)
    r, p = [torch.unsqueeze(t,dim=-1) for t in (r,p)]
    mean, var = r*(1-p)/p, r*(1-p)/(p*p)
    mean, var = torch.nn.functional.softplus(mean), torch.nn.functional.softplus(var)
    # mean, var = torch.nn.functional.softplus(r), torch.nn.functional.softplus(p)
    y = torch.cat((mean,var), axis=1)

    return y

In [ ]:
def negative_loss(pred, target):
   #https://github.com/wukevin/babel/blob/31955790709c5b18350cd6662f21e961ce74d52a/babel/loss_functions.py#L508
    mean,theta = torch.unbind(pred, dim=1)
    eps = 1e-10
    theta = torch.clamp(theta, max=1e6)
    t1 = (
        torch.lgamma(theta + eps)
        + torch.lgamma(target + 1.0)
        - torch.lgamma(target+ theta + eps)
    )
    t2 = (theta + target) * torch.log1p(mean / (theta + eps)) + (
        target * (torch.log(theta + eps) - torch.log(mean + eps))
    )
    retval = t1 + t2
    return torch.mean(retval)

In [ ]:
def negative_binomial_loss(pred, target):
    n,p = torch.unbind(pred, dim=1)
    n,p = [torch.unsqueeze(t,dim=-1) for t in (n,p)]
    nll = (
        torch.lgamma(n) 
        + torch.lgamma(target + 1)
        - torch.lgamma(n + target)
        - n * torch.log(p)
        - target * torch.log(1 - p)
    )    
    return torch.mean(nll).float()

In [ ]:
def logcosh(true, pred):
    loss = torch.log(torch.cosh(pred - true))
    return torch.sum(loss)

### Data Augmentation

## Training and Validating

In [ ]:
save_model_time = '0'
mkpath = 'model/model%s'% save_model_time
# os.makedirs(mkpath)

In [ ]:
class TrainHelper():
    '''
    Helper class that makes it a bit easier and cleaner to define the training routine
    
    '''

    def __init__(self,model,train_set,test_set,opts):
      self.model = model  # neural net

      # device agnostic code snippet
      self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      self.model.to(self.device)

      self.epochs = opts['epochs']

      if opts['opt'] == 'Adam':
        self.optimizer = torch.optim.Adam(model.parameters(), opts['lr']) # optimizer method for gradient descent
      elif opts['opt'] == 'sgd':
        self.optimizer = torch.optim.SGD(model.parameters(), opts['lr'])
      elif opts['opt'] == 'adadelta':
        self.optimizer = torch.optim.Adadelta(model.parameters(), opts['lr'])

      if opts['loss_fxn'] == 'mse':
        self.criterion = torch.nn.MSELoss()                      # loss function
      elif opts['loss_fxn'] == 'mae':
        self.criterion = torch.nn.L1Loss()
      elif opts['loss_fxn'] == 'smooth':
        self.criterion = torch.nn.SmoothL1Loss(reduction='mean')
      elif opts['loss_fxn'] == 'huber':
        self.criterion = torch.nn.SmoothL1Loss(reduction='mean', beta=0.3)
      elif opts['loss_fxn'] == 'neg':
        #self.criterion = negative_binomial_loss
        self.criterion = negative_loss
      elif opts['loss_fxn'] == 'cosh':
        self.criterion = logcosh
      elif opts['loss_fxn'] == 'poisson':
        self.criterion = torch.nn.PoissonNLLLoss(log_input=False)

      self.train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                      batch_size=opts['batch_size'],
                                                      shuffle=True)
      self.valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                                      batch_size=opts['batch_size'],
                                                      shuffle=True)
    def train(self):
      self.model.train() # put model in training mode
      for epoch in range(self.epochs):
          self.tr_loss = []
          for i, (data,labels) in tqdm_notebook(enumerate(self.train_loader),
                                                  total = len(self.train_loader)):

              data, labels = data.to(self.device),labels.to(self.device)
              self.optimizer.zero_grad()  
              outputs = self.model(data)
              loss = 0
              if opts['loss_fxn'] != 'cosh' or opts['loss_fxn'] != 'neg' or opts['loss_fxn'] != 'poisson':
                labels = labels.unsqueeze(1)
                loss = self.criterion(outputs.float(), labels.float())
              else:
                loss = self.criterion(outputs, labels.unsqueeze(dim=-1))

              loss.backward()
              self.optimizer.step()       
              self.tr_loss.append(loss.item())       
          if (epoch+1) % 5 == 0 or epoch == 0: # save the model every _ epoch
              torch.save(self.model, 'model/model{save_model_time}/net_{epoch}.pkl'.format(save_model_time=save_model_time,epoch=int((epoch+1)/5)))
              torch.save(self.model.state_dict(), 'model/model{save_model_time}/net_params_{epoch}.pkl'.format(save_model_time=save_model_time,epoch=int((epoch+1)/5)))
          
          self.test(epoch) # run through the validation set
          # print('epoch: {}, train loss: {}'.format(epoch+1, np.mean(self.tr_loss)))
          # train_loss.append(np.mean(self.tr_loss))

    def test(self,epoch):
            
      #self.model.eval()    # puts model in eval mode
      self.test_loss = []
      self.test_accuracy = []

      for i, (data, labels) in enumerate(self.valid_loader):
          
          data, labels = data.to(self.device),labels.to(self.device)
          # pass data through network
          # turn off gradient calculation to speed up calcs and reduce memory
          with torch.no_grad():
              outputs = self.model(data)
          # make our predictions and update our loss info
          # labels = labels.unsqueeze(1)
          # loss = self.criterion(outputs, labels)
          # loss = self.criterion(outputs, labels.unsqueeze(dim=-1))
          loss = 0
          if opts['loss_fxn'] == 'mse':
            labels = labels.unsqueeze(1)
            loss = self.criterion(outputs.float(), labels.float())
          else:
            loss = self.criterion(outputs, labels.unsqueeze(dim=-1))
          self.test_loss.append(loss.item())
      
      test_loss.append(np.mean(self.test_loss))
      train_loss.append(np.mean(self.tr_loss))    
      bestmodel(self.model,save_model_time,np.mean(self.test_loss)) # find best model
      print('epoch: {}, train loss: {}, test loss: {}'.format( 
      epoch+1, np.mean(self.tr_loss), np.mean(self.test_loss)))

## Testing

In [ ]:
train_X, train_y = shuffle(train_X, train_y, random_state=0) 
train_X_sub = train_X[:2000]
train_y_sub = train_y[:2000]
sub_dataset = MyDataset(train_X_sub, train_y_sub)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=True)
sub_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)

In [ ]:
def get_list_con(model, loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pred, true = [], []
    for i, (data, labels) in enumerate(loader):
      data, labels = data.to(device),labels.to(device)
    # pass data through network
    # turn off gradient calculation to speed up calcs and reduce memory
      with torch.no_grad():
          outputs = model(data)
    # make our predictions and update our loss info
      predicted = []
      for o in outputs.tolist():
        predicted.append(o[0])
      pred.extend(predicted)
      true.extend(labels.tolist())
    return true, pred

### AUC

In [ ]:
def getAUC(model):
    labels_tr, predicts_tr = get_list_cat(model, sub_loader)
    score_tr = metrics.roc_auc_score(labels_tr, predicts_tr, average='weighted')
    labels_ts, predicts_ts = get_list_cat(model, test_loader)
    score_ts = metrics.roc_auc_score(labels_ts, predicts_ts, average='weighted')
    return score_tr, score_ts

### AUPRC

In [ ]:
def getAUPRC(model):
    labels, predicts = get_list_con(model)
    auprc = average_precision_score(labels, predicts)
    return auprc

### Pearson R

In [ ]:
def getR(model):
    labels_tr, predicts_tr = get_list_con(model, sub_loader)
    corr_tr, _ = stats.pearsonr(labels_tr, predicts_tr)
    labels_ts, predicts_ts = get_list_con(model, test_loader)
    corr_ts, _ = stats.pearsonr(labels_ts, predicts_ts)
    return corr_tr, corr_ts

### Spearman R

In [ ]:
def getSR(model):
    labels_tr, predicts_tr = get_list_con(model, sub_loader)
    corr_tr, _ = stats.spearmanr(labels_tr, predicts_tr)
    labels_ts, predicts_ts = get_list_con(model, test_loader)
    corr_ts, _ = stats.spearmanr(labels_ts, predicts_ts)
    return corr_tr, corr_ts

### R2

In [ ]:
def getR2(model):
  labels_tr, predicts_tr = get_list_con(model, sub_loader)
  r2_tr = metrics.r2_score(labels_tr, predicts_tr)
  labels_ts, predicts_ts = get_list_con(model, test_loader)
  r2_ts = metrics.r2_score(labels_ts, predicts_ts)
  return r2_tr, r2_ts

### Average Percentage Change

In [ ]:
def avgDiff(model):
    labels, predicts = get_list_con(model, test_loader)
    all = []
    for i, y in enumerate(labels):
      div = y
      if y == 0:
        div = 0.0000000001
      all.append((predicts[i]-y)/div)
    all = np.array(all)
    all_abs = np.absolute(all)
    return np.mean(all_abs)

### Plot Train Verse Test Loss

In [ ]:
def pltloss(train_loss, test_loss, epoch):
    epochs = [i for i in range(epoch)]
    fig = plt.figure()
    plt.plot(epochs, train_loss, 'g', label='Training loss')
    plt.plot(epochs, test_loss, 'b', label='Testing loss')
    plt.title('Training and Testing Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

### Plot R

In [ ]:
def pltR(r_tr, r_ts, epoch):
    epochs = [i for i in range(epoch+1)][::5][1:]
    fig = plt.figure()
    plt.plot(epochs, r_tr, 'g', label='Pearson R for Training')
    plt.plot(epochs, r_ts, 'b', label='Pearson R for Testing')
    plt.title('R Score Over Time')
    plt.xlabel('Epochs')
    plt.ylabel('R')
    plt.legend()
    plt.show()

In [ ]:
def pltSR(r_tr, r_ts, epoch):
    epochs = [i for i in range(epoch+1)][::5][1:]
    fig = plt.figure()
    plt.plot(epochs, r_tr, 'g', label='Spearman R for Training')
    plt.plot(epochs, r_ts, 'b', label='Spearman R for Testing')
    plt.title('R Score Over Time')
    plt.xlabel('Epochs')
    plt.ylabel('R')
    plt.legend()
    plt.show()

In [ ]:
def pltR2(r_tr, r_ts, epoch):
    epochs = [i for i in range(epoch+1)][::5][1:]
    fig = plt.figure()
    plt.plot(epochs, r_tr, 'g', label='R2 for Training')
    plt.plot(epochs, r_ts, 'b', label='R2 for Testing')
    plt.title('R Score Over Time')
    plt.xlabel('Epochs')
    plt.ylabel('R')
    plt.legend()
    plt.show()

### Plot Predicted Verse Label

In [ ]:
def plotcomp(model, loader):
    labels, predicts = get_list_con(model, loader)
    fig = plt.figure()
    plt.scatter(labels, predicts)
    l = max(max(predicts), max(labels))
    s = min(min(predicts), min(labels))
    plt.plot([s, l], [s, l], color = 'black', linewidth = 1)
    plt.title('Actual Values vs Predicted Values')
    plt.ylabel('Predicted Values')
    plt.xlabel('Actual Values')
    plt.xlim(s, l)
    plt.ylim(s, l)
    plt.show()

## Models

### CNN

In [ ]:
# class CNN(nn.Module):
#     def __init__(self, input_size):
#         """
#         init convolution and activation layers
#         Args:
#         x: (Nx1x2004)
#         class: 

#         """
#         super(CNN, self).__init__() 
        
#         self.conv1 = torch.nn.Conv1d(4, 32, 7)
#         # self.relu = torch.nn.ReLU()
#         # self.tanh = torch.nn.Tanh()
#         self.gelu = torch.nn.GELU()
#         # self.soft = torch.nn.Softplus()
#         self.conv2 = torch.nn.Conv1d(32, 64, 7)
#         self.conv3 = torch.nn.Conv1d(64, 64, 7)
#         self.pool = torch.nn.MaxPool1d(7)
#         # self.fc1 = torch.nn.Linear(64*200, 64*200)
#         # self.fc2 = torch.nn.Linear(64*200, 1)
#         # self.fc1 = torch.nn.Linear(64*20, 1)
#         self.fc1 = torch.nn.Linear(12608,1)
#         #self.gru = nn.GRU(input_size=32, hidden_size=10, num_layers=2, batch_first=True, dropout=0, bidirectional=True)
#         #self.gru = nn.GRU(input_size=47, hidden_size=10, num_layers=2, batch_first=True, dropout=0, bidirectional=True)
#         #self.gru = nn.GRU(input_size=58, hidden_size=10, num_layers=2, batch_first=True, dropout=0, bidirectional=True)
#         self.gru = nn.GRU(input_size=197, hidden_size=10, num_layers=2, batch_first=True, dropout=0, bidirectional=True)
#         # self.batch = nn.BatchNorm1d(32)
#         # self.batch1 = nn.BatchNorm1d(64)
#         self.dropout = nn.Dropout(p=0.2, inplace=True)

#     def forward(self, x):
#         """
#         forward function describes how input tensor is transformed to output tensor
#         Args:
            
#         """
#         x = self.conv1(x)
#         x = self.gelu(x)
#         x = self.pool(x)
#         x = self.dropout(x)
#         x = self.conv2(x)
#         x = self.gelu(x)
#         x = self.pool(x)
#         x = self.dropout(x)
#         x = self.conv3(x)
#         x = self.gelu(x)
#         #x = self.gru(x)[0]
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         # x = self.fc2(x)
#         return x

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size):
        """
        init convolution and activation layers
        Args:
        x: 
        class: 

        """
        super(CNN, self).__init__() 
        
        self.conv1 = torch.nn.Conv1d(input_size[0], 32, 4)
        self.relu = torch.nn.ReLU()
        self.tanh = torch.nn.Tanh()
        self.gelu = torch.nn.GELU()
        self.conv2 = torch.nn.Conv1d(32, 64, 4)
        self.conv3 = torch.nn.Conv1d(64, 64, 4)
        self.pool = torch.nn.MaxPool1d(4)
        self.fc1 = torch.nn.Linear(64*20, 1)
        self.fc2 = torch.nn.Linear(200*64, 1)
        if input_size[1] == 601:
          self.gru = nn.GRU(input_size=34, hidden_size=10, num_layers=2, batch_first=True, dropout=0, bidirectional=True)
        elif input_size[1] == 1001:
          self.gru = nn.GRU(input_size=58, hidden_size=10, num_layers=2, batch_first=True, dropout=0, bidirectional=True)
        elif input_size[1] == 2001:
          self.gru = nn.GRU(input_size=121, hidden_size=10, num_layers=2, batch_first=True, dropout=0, bidirectional=True)
        #self.gru = nn.GRU(input_size=621, hidden_size=100, num_layers=2, batch_first=True, dropout=0, bidirectional=True)
        self.dropout = nn.Dropout(p=0.5, inplace=True)

    def forward(self, x):
        """
        forward function describes how input tensor is transformed to output tensor
        Args:
            
        """
        x = self.conv1(x)
        x = self.gelu(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.gelu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.gelu(x)
        x = self.gru(x)[0]
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        #x = self.fc2(x)
        #x = negative_binomial_dist(x) # convert to params of distribution (n and p)
        #x = negative_binomial_layer(x)
        return x

In [ ]:
# class CNN(nn.Module):
#     def __init__(self, input_size):
#         """
#         init convolution and activation layers
#         Args:
#         x: (Nx1x2004)
#         class: 

#         """
#         super(CNN, self).__init__() 
        
#         self.gru = nn.GRU(input_size=input_size[1], hidden_size=200, num_layers=3, batch_first=True, dropout=0.8, bidirectional=True)
#         self.fc1 = nn.Linear(1600,1)

#     def forward(self, x):
#         """
#         forward function describes how input tensor is transformed to output tensor
#         Args:
            
#         """
#         x = self.gru(x)[0]
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         return x

In [ ]:
cnn = CNN(train_X.shape[1:])
cnn

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 25,
    'batch_size': 100,
    'loss_fxn': 'mse',
    'opt': 'Adam'
}

In [ ]:
test_loss, train_loss = [], []
CNNTrainer = TrainHelper(model = cnn,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
CNNTrainer.train()

#### Check for Output

In [ ]:
r_list_ts = []
r_list_tr = []
for num in range(opts['epochs']//5):
  cnn.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  cnn.cuda()
  tr, ts = getR(cnn)
  r_list_ts.append(ts)
  r_list_tr.append(tr)

In [ ]:
r_list_ps = []
r_list_pr = []
for num in range(opts['epochs']//5):
  cnn.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  cnn.cuda()
  tr, ts = getSR(cnn)
  r_list_ps.append(ts)
  r_list_pr.append(tr)

In [ ]:
r_list_rs = []
r_list_rr = []
for num in range(opts['epochs']//5):
  cnn.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  cnn.cuda()
  tr, ts = getR2(cnn)
  r_list_rs.append(ts)
  r_list_rr.append(tr)

#### Mutation

In [ ]:
''' get individual value'''
seed_everything()
cnn.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_12.pkl'))
dummy, pred = get_list_con(cnn, mut_loader)

In [ ]:
mut_X = torch.from_numpy(np.array([onehot(sample)]))
mut_Y = torch.from_numpy(np.array([pred]))
mut_dataset = MyDataset(mut_X, mut_Y)
mut_loader = torch.utils.data.DataLoader(mut_dataset, batch_size=1, shuffle=False)
# reverse(mut_dataset.data[0])

In [ ]:
cnn.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_12.pkl'))
cnn.cuda()
seed_everything()
w1 = weight_matrix(cnn)
# np.save('weight_sst_new', w1)

In [ ]:
# seed_everything()
# heat(mut_loader, cnn)

#### Plots

In [ ]:
pltloss(train_loss, test_loss, opts['epochs'])

In [ ]:
pltR(r_list_tr, r_list_ts, opts['epochs'])

In [ ]:
pltSR(r_list_pr, r_list_ps, opts['epochs'])

In [ ]:
pltR2(r_list_rr, r_list_rs, opts['epochs'])

In [ ]:
seed_everything()
plotcomp(cnn, test_loader)

In [ ]:
seed_everything()
plotcomp(cnn, sub_loader)

In [ ]:
''' extract cnn layer '''
# layer_shape = cnn.conv1.weight.shape
# for i in range(layer_shape[0]):
#     fig = plt.figure(figsize = (5, 5))
#     plt.imshow(cnn.conv1.weight[i].detach().numpy(), cmap = 'Blues')
#     fig.subplots_adjust(right=0.85)
#     cax = plt.axes([0.9, 0.1, 0.075, 0.8])
#     plt.colorbar(cax=cax)
#     plt.show()

### Basenji

In [ ]:
''' original version based on https://github.com/calico/basenji/blob/master/manuscripts/genome_research2018/params.txt '''
class Besenji(nn.Module):
    def __init__(self, input_size):
        """
        init convolution and activation layers
        Args:
        class: 

        """
        super(Besenji, self).__init__() 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.05)
        self.dropout1 = nn.Dropout(p=0.1)
        self.batch6 = nn.BatchNorm1d(108)

        self.conv1 = nn.Conv1d(input_size[0], 312, kernel_size=22)
        self.batch1 = nn.BatchNorm1d(312)

        self.conv2 = nn.Conv1d(312, 368, kernel_size=1)
        self.pool2 = nn.MaxPool1d(2)
        self.batch2 = nn.BatchNorm1d(368)

        self.conv3 = nn.Conv1d(368, 435, kernel_size=6)
        self.pool3 = nn.MaxPool1d(4)
        self.batch3 = nn.BatchNorm1d(435)

        self.conv4 = nn.Conv1d(435, 607, kernel_size=6)
        self.pool4 = nn.MaxPool1d(4)
        self.batch4 = nn.BatchNorm1d(607)
        
        self.conv5 = nn.Conv1d(607, 717, kernel_size=3)
        self.batch5 = nn.BatchNorm1d(717)

        self.conv6 = nn.Conv1d(717, 108, kernel_size=3, dilation=2)

        self.conv7 = nn.Conv1d(108, 108, kernel_size=3, dilation=4)

        self.conv8 = nn.Conv1d(108, 108, kernel_size=3, dilation=8)

        self.conv9 = nn.Conv1d(108, 108, kernel_size=3, dilation=16)
        
        self.conv10 = nn.Conv1d(108, 108, kernel_size=3, dilation=32)

        self.conv11 = nn.Conv1d(108, 108, kernel_size=3, dilation=64)

        self.conv12 = nn.Conv1d(108, 1365, kernel_size=1)
        self.batch12 = nn.BatchNorm1d(1365)

        self.conv13 = nn.Conv1d(1365, 1, kernel_size=1)
        self.fc1 = torch.nn.Linear(15, 1)

    def forward(self, x):
        x = self.batch1(self.conv1(x))
        x = self.relu(x)
        x = self.dropout(x)

        x = self.relu(self.batch2(self.conv2(x)))
        x = self.pool2(x)
        x = self.dropout(x)

        x = self.relu(self.batch3(self.conv3(x)))
        x = self.pool3(x)
        x = self.dropout(x)

        x = self.relu(self.batch4(self.conv4(x)))
        x = self.pool4(x)
        x = self.dropout(x)

        x = self.relu(self.batch5(self.conv5(x)))
        x = self.dropout(x)

        x = self.relu(self.batch6(self.conv6(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch6(self.conv7(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch6(self.conv8(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch6(self.conv9(x)))
        x = self.dropout1(x)
        
        x = self.relu(self.batch6(self.conv10(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch6(self.conv11(x)))
        x = self.dropout1(x)

        x = self.relu(self.batch12(self.conv12(x)))
        x = self.dropout(x)
        
        x = self.conv13(x)
        
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        return x

In [ ]:
''' modified version '''
class Besenji(nn.Module):
    def __init__(self, input_size):
        """
        init convolution and activation layers
        Args:
        class: 

        """
        super(Besenji, self).__init__() 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.05)
        self.dropout1 = nn.Dropout(p=0.1)
        self.batch6 = nn.BatchNorm1d(108)

        self.conv1 = nn.Conv1d(4, 312, kernel_size=2)
        self.batch1 = nn.BatchNorm1d(312)

        self.conv2 = nn.Conv1d(312, 368, kernel_size=1)
        self.pool2 = nn.MaxPool1d(2)
        self.batch2 = nn.BatchNorm1d(368)

        self.conv3 = nn.Conv1d(368, 435, kernel_size=2)
        self.pool3 = nn.MaxPool1d(2)
        self.batch3 = nn.BatchNorm1d(435)

        self.conv4 = nn.Conv1d(435, 607, kernel_size=2)
        self.pool4 = nn.MaxPool1d(2)
        self.batch4 = nn.BatchNorm1d(607)
        
        self.conv5 = nn.Conv1d(607, 717, kernel_size=3)
        self.batch5 = nn.BatchNorm1d(717)

        self.conv6 = nn.Conv1d(717, 108, kernel_size=3, dilation=2)

        self.conv7 = nn.Conv1d(108, 108, kernel_size=3, dilation=4)

        self.conv8 = nn.Conv1d(108, 108, kernel_size=3, dilation=8)

        self.conv9 = nn.Conv1d(108, 108, kernel_size=3, dilation=16)
        
        self.conv10 = nn.Conv1d(108, 108, kernel_size=3, dilation=32)

        self.conv11 = nn.Conv1d(108, 108, kernel_size=3, dilation=64)

        self.conv12 = nn.Conv1d(108, 1365, kernel_size=1)
        self.batch12 = nn.BatchNorm1d(1365)

        self.conv13 = nn.Conv1d(1365, 1, kernel_size=1)
        self.fc1 = torch.nn.Linear(12, 1)
        #self.fc2 = torch.nn.Linear(68, 1)
        #self.fc2 = torch.nn.Linear(93, 1)
        #self.fc2 = torch.nn.Linear(118, 1)
        self.fc2 = torch.nn.Linear(243, 1)


    def forward(self, x):
        x = self.batch1(self.conv1(x))
        x = self.relu(x)
        x = self.dropout(x)

        x = self.relu(self.batch2(self.conv2(x)))
        x = self.pool2(x)
        x = self.dropout(x)

        x = self.relu(self.batch3(self.conv3(x)))
        x = self.pool3(x)
        x = self.dropout(x)

        x = self.relu(self.batch4(self.conv4(x)))
        x = self.pool4(x)
        x = self.dropout(x)
        
        x = self.relu(self.batch5(self.conv5(x)))
        x = self.dropout(x)

        x = self.relu(self.batch6(self.conv6(x)))
        x = self.dropout1(x)
        # print(x.size())

        # x = self.relu(self.batch6(self.conv7(x)))
        # x = self.dropout1(x)
        # # print(x.size())

        # x = self.relu(self.batch6(self.conv8(x)))
        # x = self.dropout1(x)
        # # print(x.size())

        # x = self.relu(self.batch6(self.conv9(x)))
        # x = self.dropout1(x)
        # print(x.size())
        
        # x = self.relu(self.batch6(self.conv10(x)))
        # x = self.dropout1(x)
        # print(x.size())

        # x = self.relu(self.batch6(self.conv11(x)))
        # x = self.dropout1(x)
        # print(x.size())

        x = self.relu(self.batch12(self.conv12(x)))
        x = self.dropout(x)
        # print(x.size())
      
        x = self.conv13(x)
        x = torch.flatten(x, 1)
        # x = self.fc1(x)
        x = self.fc2(x)
        # print(x.size())
        # assert 0

        return x

In [ ]:
basenji = Besenji(train_X.shape[1:])
basenji

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 50,
    'batch_size': 100,
    'loss_fxn': 'smooth',
    'opt': 'sgd'
}

In [ ]:
test_loss, train_loss = [], []
BasenjiTrainer = TrainHelper(model = basenji,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
BasenjiTrainer.train()

#### Check for Output

In [ ]:
r_list_ts = []
r_list_tr = []
for num in range(opts['epochs']//5):
  basenji.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  basenji.cuda()
  tr, ts = getR(basenji)
  r_list_ts.append(ts)
  r_list_tr.append(tr)

#### Plots

In [ ]:
pltloss(train_loss, test_loss, opts['epochs'])

In [ ]:
pltR(r_list_tr, r_list_ts, opts['epochs'])

In [ ]:
plotcomp(basenji,test_loader)

### VGG

In [ ]:
class VGG16(nn.Module):
    def __init__(self, input_size):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv1d(input_size[0], 64, kernel_size=4, padding=1)
        self.conv1_2 = nn.Conv1d(64, 64, kernel_size=4, padding=1)

        self.conv2_1 = nn.Conv1d(64, 128, kernel_size=4, padding=1)
        self.conv2_2 = nn.Conv1d(128, 128, kernel_size=4, padding=1)

        self.conv3_1 = nn.Conv1d(128, 256, kernel_size=4, padding=1)
        self.conv3_2 = nn.Conv1d(256, 256, kernel_size=4, padding=1)
        self.conv3_3 = nn.Conv1d(256, 256, kernel_size=4, padding=1)

        self.conv4_1 = nn.Conv1d(256, 512, kernel_size=4, padding=1)
        self.conv4_2 = nn.Conv1d(512, 512, kernel_size=4, padding=1)
        self.conv4_3 = nn.Conv1d(512, 512, kernel_size=4, padding=1)

        self.conv5_1 = nn.Conv1d(512, 512, kernel_size=4, padding=1)
        self.conv5_2 = nn.Conv1d(512, 512, kernel_size=4, padding=1)
        self.conv5_3 = nn.Conv1d(512, 512, kernel_size=4, padding=1)

        #self.gru = nn.GRU(input_size=18, hidden_size=10, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool1d(4, 4)

        # fully conected layers
        #self.fc6 = nn.Linear(512*18, 1000)
        #self.fc6 = nn.Linear(512*20, 1000)
        self.fc6 = nn.Linear(512*9, 1000)
        self.fc7 = nn.Linear(1000, 1000)
        self.fc8 = nn.Linear(1000, 1)

    def forward(self, x):
        x = self.conv1_1(x)
        x = self.relu(x)
        x = self.conv1_2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2_1(x)
        x = self.relu(x)
        x = self.conv2_2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv3_1(x)
        x = self.relu(x)
        x = self.conv3_2(x)
        x = self.relu(x)
        x = self.conv3_3(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv4_1(x)
        x = self.relu(x)
        x = self.conv4_2(x)
        x = self.relu(x)
        x = self.conv4_3(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.relu(self.conv5_1(x))
        x = self.relu(self.conv5_2(x))
        x = self.relu(self.conv5_3(x))
        x = self.pool(x)
        # print(x.size())
        #x = self.gru(x)[0]
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc7(x)
        # x = self.log(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc8(x)

        return x

In [ ]:
vgg = VGG16(train_X.shape[1:])
vgg

In [ ]:
opts = {
    'lr': 1e-3,
    'epochs': 25,
    'batch_size': 100,
    'loss_fxn': 'mse',
    'opt': 'sgd'
}

In [ ]:
test_loss, train_loss = [], []
VGGTrainer = TrainHelper(model = vgg,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
VGGTrainer.train()

#### Check for Output

In [ ]:
r_list_ts = []
r_list_tr = []
for num in range(opts['epochs']//5):
  vgg.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  vgg.cuda()
  tr, ts = getR(vgg)
  r_list_ts.append(ts)
  r_list_tr.append(tr)

#### Plots

In [ ]:
pltloss(train_loss, test_loss, opts['epochs'])

In [ ]:
pltR(r_list_tr, r_list_ts, opts['epochs'])

In [ ]:
plotcomp(vgg,test_loader)

### Char Plant

In [ ]:
class CHAR(nn.Module):
    def __init__(self, input_size):
        """
        init convolution and activation layers
        Args:
        class: 

        """
        super(CHAR, self).__init__() 
        
        self.conv1 = torch.nn.Conv1d(input_size[0], 200, 19, padding="same")
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv1d(200, 100, 11, padding="same")
        self.fc1 = torch.nn.Linear(62500, 200)
        self.fc2 = torch.nn.Linear(200, 1)
        self.dropout = nn.Dropout(p=0.6)
        self.gru = nn.GRU(input_size=62, hidden_size=31, num_layers=2, batch_first=True, dropout=0.6, bidirectional=True)
        self.pool = nn.MaxPool1d(4, 4)

    def forward(self, x):
        """
        forward function describes how input tensor is transformed to output tensor
        Args:
            
        """
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.pool(x)
        #print(x.size())
        #x = self.gru(x)[0]

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)


        return x

In [ ]:
char = CHAR(train_X.shape[1:])
char

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 50,
    'batch_size': 100,
    'loss_fxn': 'mse',
    'opt': 'Adam'
}

In [ ]:
test_loss, train_loss = [], []
CharTrainer = TrainHelper(model = char,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
CharTrainer.train()

#### Check for Output

In [ ]:
r_list_ts = []
r_list_tr = []
for num in range(opts['epochs']//5):
  char.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  char.cuda()
  tr, ts = getR(char)
  r_list_ts.append(ts)
  r_list_tr.append(tr)

#### Plots

In [ ]:
pltR(r_list_tr, r_list_ts, opts['epochs'])

In [ ]:
plotcomp(char,test_loader)

### AlexNet

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, input_size):
        """
        init convolution and activation layers
        Args:
        class: 

        """
        super(AlexNet, self).__init__() 
        
        self.features = nn.Sequential(
            nn.Conv1d(4, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
        )

        self.avgpool = nn.AdaptiveAvgPool1d(6)
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6, 256 * 4),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256 * 4, 256 * 4),
            nn.ReLU(inplace=True),
            nn.Linear(256 * 4, 1),
        )

    def forward(self, x):
        """
        forward function describes how input tensor is transformed to output tensor
        Args:
            
        """
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
alex = AlexNet(train_X.shape[1:])
alex

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 50,
    'batch_size': 256,
    'loss_fxn': 'mse',
    'opt': 'Adam'
}

In [ ]:
test_loss, train_loss = [], []
alexTrainer = TrainHelper(model = alex,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
alexTrainer.train()

#### Check for Output

In [ ]:
r_list_ts = []
r_list_tr = []
for num in range(opts['epochs']//5):
  alex.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  alex.cuda()
  tr, ts = getR(alex)
  r_list_ts.append(ts)
  r_list_tr.append(tr)

#### Plots

In [ ]:
pltR(r_list_tr, r_list_ts, opts['epochs'])

In [ ]:
plotcomp(alex,test_loader)

### LeNet5

In [ ]:
class LeNet5(nn.Module):

    def __init__(self, input_size):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv1d(in_channels=input_size[0], out_channels=6, kernel_size=5, stride=1),
            nn.GELU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.GELU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.GELU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=299160, out_features=100),
            nn.GELU(),
            nn.Linear(in_features=100, out_features=1),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
      
        return x

In [ ]:
lenet = LeNet5(train_X.shape[1:])
lenet

In [ ]:
opts = {
    'lr': 2e-5,
    'epochs': 50,
    'batch_size': 100,
    'loss_fxn': 'mse',
    'opt': 'Adam'
}

In [ ]:
test_loss, train_loss = [], []
lenetTrainer = TrainHelper(model = lenet,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
lenetTrainer.train()

#### Check for Output

In [ ]:
r_list_ts = []
r_list_tr = []
for num in range(opts['epochs']//5):
  lenet.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  lenet.cuda()
  tr, ts = getR(lenet)
  r_list_ts.append(ts)
  r_list_tr.append(tr)

#### Plots

In [ ]:
pltR(r_list_tr, r_list_ts, opts['epochs'])

In [ ]:
plotcomp(lenet,sub_loader)

### ResNet

In [ ]:
import torch.utils.model_zoo as model_zoo

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

def conv5x5(in_planes, out_planes, stride=1):
    return nn.Conv1d(in_planes, out_planes, kernel_size=5, stride=stride,
                     padding=1, bias=False)

def conv7x7(in_planes, out_planes, stride=1):
    return nn.Conv1d(in_planes, out_planes, kernel_size=7, stride=stride,
                     padding=1, bias=False)



class BasicBlock3x3_1(nn.Module):
    expansion = 1

    def __init__(self, inplanes3_1, planes, stride=1, downsample=None):
        super(BasicBlock3x3_1, self).__init__()
        self.conv1 = conv3x3(inplanes3_1, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class BasicBlock3x3_2(nn.Module):
    expansion = 1

    def __init__(self, inplanes3_2, planes, stride=1, downsample=None):
        super(BasicBlock3x3_2, self).__init__()
        self.conv1 = conv3x3(inplanes3_2, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class BasicBlock3x3_3(nn.Module):
    expansion = 1

    def __init__(self, inplanes3_3, planes, stride=1, downsample=None):
        super(BasicBlock3x3_3, self).__init__()
        self.conv1 = conv3x3(inplanes3_3, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class MSResNet(nn.Module):
    def __init__(self, input_channel, layers=[2, 2, 2, 2], num_classes=1):
        self.inplanes3_1 = 64
        self.inplanes3_2 = 64
        self.inplanes3_3 = 64

        super(MSResNet, self).__init__()

        self.conv1 = nn.Conv1d(input_channel, 64, kernel_size=5, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer3x3_11 = self._make_layer3_1(BasicBlock3x3_1, 64, layers[0], stride=2)
        self.layer3x3_12 = self._make_layer3_1(BasicBlock3x3_1, 128, layers[1], stride=2)
        self.layer3x3_13 = self._make_layer3_1(BasicBlock3x3_1, 256, layers[2], stride=2)
        #self.layer3x3_14 = self._make_layer3_1(BasicBlock3x3_1, 512, layers[3], stride=2)

        # maxplooing kernel size: 16, 11, 6
        self.maxpool3_1 = nn.AdaptiveAvgPool1d(3)
        #(kernel_size=3, stride=1, padding=0)

        self.layer3x3_21 = self._make_layer3_2(BasicBlock3x3_2, 64, layers[0], stride=2)
        self.layer3x3_22 = self._make_layer3_2(BasicBlock3x3_2, 128, layers[1], stride=2)
        self.layer3x3_23 = self._make_layer3_2(BasicBlock3x3_2, 256, layers[2], stride=2)
        #self.layer3x3_24 = self._make_layer3_2(BasicBlock3x3_2, 512, layers[3], stride=2)

        # maxplooing kernel size: 16, 11, 6
        self.maxpool3_2 = nn.AdaptiveAvgPool1d(3)
        #(kernel_size=3, stride=1, padding=0)

        self.layer3x3_31 = self._make_layer3_3(BasicBlock3x3_3, 64, layers[0], stride=2)
        self.layer3x3_32 = self._make_layer3_3(BasicBlock3x3_3, 128, layers[1], stride=2)
        self.layer3x3_33 = self._make_layer3_3(BasicBlock3x3_3, 256, layers[2], stride=2)
        #self.layer3x3_34 = self._make_layer3_3(BasicBlock3x3_3, 512, layers[3], stride=2)

        # maxplooing kernel size: 16, 11, 6
        self.maxpool3_3 = nn.AdaptiveAvgPool1d(3)
        #(kernel_size=3, stride=1, padding=0)

        self.drop = nn.Dropout(p=0.2)
        #self.fc = nn.Linear(256*51, num_classes)
        self.fc = nn.Linear(256*9, num_classes)


    def _make_layer3_1(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes3_1 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes3_1, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes3_1, planes, stride, downsample))
        self.inplanes3_1 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes3_1, planes))

        return nn.Sequential(*layers)

    def _make_layer3_2(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes3_2 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes3_2, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes3_2, planes, stride, downsample))
        self.inplanes3_2 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes3_2, planes))

        return nn.Sequential(*layers)


    def _make_layer3_3(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes3_3 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes3_3, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes3_3, planes, stride, downsample))
        self.inplanes3_3 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes3_3, planes))

        return nn.Sequential(*layers)


    def forward(self, x0):
        x0 = self.conv1(x0)
        x0 = self.bn1(x0)
        x0 = self.relu(x0)
        x0 = self.maxpool(x0)

        x = self.layer3x3_11(x0)
        x = self.layer3x3_12(x)
        x = self.drop(x)
        x = self.layer3x3_13(x)
        # x = self.layer3x3_14(x)
        x = self.maxpool3_1(x)

        y = self.layer3x3_21(x0)
        y = self.layer3x3_22(y)
        y = self.drop(y)
        y = self.layer3x3_23(y)
        # y = self.layer3x3_24(y)
        y = self.maxpool3_2(y)

        z = self.layer3x3_31(x0)
        z = self.layer3x3_32(z)
        z = self.drop(z)
        z = self.layer3x3_33(z)
        # z = self.layer3x3_34(z)
        z = self.maxpool3_3(z)

        out = torch.cat([x, y, z], dim=1)
        out = torch.flatten(out, 1)

        out = self.fc(out)
  

        return out

In [ ]:
resNet = MSResNet(4)
resNet

In [ ]:
''' fastai ver resnet '''
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.vision.models.xresnet import *
from fastai.metrics import *
from fastai.layers import *
resNet = xresnet18(c_in=4, n_out=1, ndim=1, pretrained=False)

In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 15,
    'batch_size': 100,
    'loss_fxn': 'mse',
    'opt': 'Adam'
}

In [ ]:
test_loss, train_loss = [], []
resnetTrainer = TrainHelper(model = resNet,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
resnetTrainer.train()

#### Check for Output

In [ ]:
r_list_ts = []
r_list_tr = []
for num in range(opts['epochs']//5):
  resNet.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  resNet.cuda()
  tr, ts = getR(resNet)
  r_list_ts.append(ts)
  r_list_tr.append(tr)

In [ ]:
r_list_rs = []
r_list_rr = []
for num in range(opts['epochs']//5):
  resNet.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  resNet.cuda()
  tr, ts = getR2(resNet)
  r_list_rs.append(ts)
  r_list_rr.append(tr)

In [ ]:
r_list_ps = []
r_list_pr = []
for num in range(opts['epochs']//5):
  resNet.load_state_dict(torch.load('model/model'+save_model_time+'/net_params_'+str(num)+'.pkl'))
  resNet.cuda()
  tr, ts = getSR(resNet)
  r_list_ps.append(ts)
  r_list_pr.append(tr)

#### Plots

In [ ]:
pltloss(train_loss, test_loss, opts['epochs'])

In [ ]:
pltR(r_list_tr, r_list_ts, opts['epochs'])

In [ ]:
pltSR(r_list_pr, r_list_ps, opts['epochs'])

In [ ]:
pltR2(r_list_rr, r_list_rs, opts['epochs'])

In [ ]:
plotcomp(resNet,test_loader)